In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# image libraries
import cv2

In [2]:
# directories
# train_img_path = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train'
# test_img_path = '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test'

train_img_path = 'artifact/train'
test_img_path = 'artifact/test'

w,h = 200,200
batch_size = 32

train_data_gen = ImageDataGenerator(
    rescale = 1.0/255,
    horizontal_flip = True,
    rotation_range = 20,
    zoom_range=0.2
)

test_data_gen = ImageDataGenerator(rescale = 1.0/255)

train_Dataset = train_data_gen.flow_from_directory(
    train_img_path,
    target_size = (w,h),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True
)

test_Dataset = test_data_gen.flow_from_directory(
    test_img_path,
    target_size = (w,h),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = False
)

Found 882752 images belonging to 2 classes.
Found 220694 images belonging to 2 classes.


In [3]:
train_Dataset.class_indices

{'FAKE': 0, 'REAL': 1}

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained ResNet50 model without the top (classification) layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Create a new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with 1 neuron (binary classification)
])

# Freeze the base model layers 
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# load best weights to model
model.load_weights('./checkpoints/my_checkpoint_resnet_databal_dataaug_20epoch')
from sklearn.metrics import accuracy_score

# make prediction
predictions = model.predict(test_Dataset)
binary_predictions = (predictions > 0.5).astype(int)

accuracy = accuracy_score(test_Dataset.classes, binary_predictions)
print(f'Accuracy: {accuracy*100:.2f}%')


6897/6897 [==============================] - 270s 38ms/step
Accuracy: 64.72%


In [11]:
from sklearn.metrics import confusion_matrix, classification_report

# Assuming you have true labels (ground truth)
true_labels = test_Dataset.classes

# Generate a confusion matrix
# sns.heatmap(confusion_matrix(true_labels, binary_predictions), annot=True)

# Print classification report
print(classification_report(true_labels, binary_predictions))

conf_matrix = confusion_matrix(true_labels, binary_predictions)
print(conf_matrix)

# plt.title('Confusion Matrix')
# plt.xlabel('Predicted')
# plt.ylabel('Actual Classes')
# plt.show()

              precision    recall  f1-score   support

           0       0.57      0.61      0.59     90962
           1       0.71      0.67      0.69    129732

    accuracy                           0.65    220694
   macro avg       0.64      0.64      0.64    220694
weighted avg       0.65      0.65      0.65    220694

[[55852 35110]
 [42750 86982]]
